In [1]:
'''Get S&P500 Constituents Data'''
import pandas as pd
import yfinance as yf

# Get the list of S&P 500 constituents
# You can get this data from a static list or by scraping a website such as Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(url)
sp500_symbols = sp500_table[0]['Symbol'].tolist()


In [2]:
'''Download Historical Data'''
import yfinance as yf

# Define the date range
start_date = '2018-01-01'
end_date = '2023-01-01'

# Download historical data for all S&P 500 constituents
data = yf.download(sp500_symbols, start=start_date, end=end_date)['Adj Close']


[********************  42%%                      ]  213 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2018-01-01 -> 2023-01-01)


[*********************100%%**********************]  503 of 503 completed

6 Failed downloads:
['SOLV', 'KVUE', 'GEV', 'VLTO']: YFChartError("%ticker%: Data doesn't exist for startDate = 1514782800, endDate = 1672549200")
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2018-01-01 -> 2023-01-01)')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


In [3]:
'''Calcualte Returns'''
# Calculate daily returns
returns = data.pct_change().dropna()


C:\Users\MP\AppData\Local\Temp\ipykernel_10292\2448819539.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()


In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Get S&P 500 constituents
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(url)
sp500_symbols = sp500_table[0]['Symbol'].tolist()

# Download historical data
start_date = '2018-01-01'
end_date = '2023-01-01'
data = yf.download(sp500_symbols, start=start_date, end=end_date)['Adj Close']

# Drop tickers with too many missing values
missing_threshold = 0.1  # Drop tickers with more than 10% missing data
data = data.dropna(thresh=int(data.shape[0] * (1 - missing_threshold)), axis=1)

# Handle remaining missing data: fill forward, then fill backward
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True)
#data.fillna().ffill(inplace=True)
#data.fillna().bfill(inplace=True)

# Ensure no inf or nan values are present
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Check if the data is empty after cleaning
if data.empty:
    raise ValueError("Data is empty after cleaning. Please check the data source and handling steps.")

# Calculate daily returns
returns = data.pct_change().dropna()

# Check if the returns DataFrame is empty
if returns.empty:
    raise ValueError("Returns data is empty after calculating percentage changes. Please check the data source and handling steps.")

[**********************91%%******************    ]  459 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2018-01-01 -> 2023-01-01)


[*********************100%%**********************]  503 of 503 completed

6 Failed downloads:
['GEV', 'VLTO', 'KVUE', 'SOLV']: YFChartError("%ticker%: Data doesn't exist for startDate = 1514782800, endDate = 1672549200")
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2018-01-01 -> 2023-01-01)')
C:\Users\MP\AppData\Local\Temp\ipykernel_65260\2070158949.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
C:\Users\MP\AppData\Local\Temp\ipykernel_65260\2070158949.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)


In [9]:
# Function to perform the cointegration test
def find_cointegrated_pairs(data, significance_level=0.05):
    n = data.shape[1]
    if n < 2:
        raise ValueError("Not enough data to perform cointegration test.")
    p_value_matrix = np.zeros((n, n))
    keys = data.columns
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            stock1 = data[keys[i]]
            stock2 = data[keys[j]]
            result = coint(stock1, stock2)
            p_value = result[1]
            p_value_matrix[i, j] = p_value
            if p_value < significance_level:
                pairs.append((keys[i], keys[j]))
    return p_value_matrix, pairs

# Find cointegrated pairs
p_value_matrix, pairs = find_cointegrated_pairs(data)

# Perform PCA for dimensionality reduction
pca = PCA(n_components=2)
returns_pca = pca.fit_transform(returns.T)

# Perform KMeans clustering
kmeans = KMeans(n_clusters=10)
kmeans.fit(returns_pca)
labels = kmeans.labels_

# Visualize the clusters
plt.figure(figsize=(10, 6))
plt.scatter(returns_pca[:, 0], returns_pca[:, 1], c=labels, cmap='viridis')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Clustering of S&P 500 Stocks Based on Returns')
plt.show()

# Function to get pairs from clusters
def get_pairs_from_clusters(labels, data, significance_level=0.05):
    unique_labels = np.unique(labels)
    pairs = []
    for label in unique_labels:
        cluster_stocks = data.columns[labels == label]
        cluster_data = data[cluster_stocks]
        if cluster_data.shape[1] < 2:
            continue  # Skip clusters with less than 2 stocks
        _, cluster_pairs = find_cointegrated_pairs(cluster_data, significance_level)
        pairs.extend(cluster_pairs)
    return pairs

# Get pairs from clusters
clustered_pairs = get_pairs_from_clusters(labels, data)

print(f"Identified pairs: {clustered_pairs}")


KeyboardInterrupt: 